In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
path = r'C:/Users/Tinatin/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe'

In [3]:
urls = ['https://www.psychologytoday.com/us/therapists/african-american/ny/new-york/345502?sid=5eee3e019ee48&ref=9&tr=ResultsPhoto', 'https://www.psychologytoday.com/us/therapists/ny/new-york/411234?sid=5eebcea19c07b&ref=1&tr=ResultsPhoto', 'https://www.psychologytoday.com/us/therapists/ny/new-york/103681?sid=5eebcea19c07b&ref=4&tr=ResultsPhoto', 'https://www.psychologytoday.com/us/therapists/ny/new-york/470297?sid=5eebcea19c07b&ref=3&tr=ResultsName']

In [4]:
final_df = pd.DataFrame(columns=['statement', 'name', 'verified', 'degree', 'addr', 'city', 'state', 'zip', 'phone', 'cost_per_ses','sliding_scale','pay_by','accepted_insurance', 'specs', 'issues', 'mental_health', 'sexuality', 'client_focus_age', 'client_focus_communities', 'treatment_approach_types_of_ther', 'treatment_approach_modality', 'additional_creds'])

### to-do: scrape information about whether they offer video sessions

In [12]:
def scraper(urls, path):
    global final_df
    #iterate over urls
    for url in urls:
        row_results = {}
        driver = Chrome(executable_path= path)
        try:
            driver.get(url)         #open the url in selenium
        except:
            print ('bad url!')
        soup = BeautifulSoup(driver.page_source,'html.parser') 
        
        #wait for the webpage to load
        time.sleep(10)
        #row_results['full_address'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = "address-data").text.split('\n') if item) if item])
        try:   
            row_results['state'] = soup.find(itemprop = "addressRegion").text
        except:
            print('state field missing')
            row_results['state'] = 'missing'
        try:
            row_results['addr'] = [item for item in (item.strip() for item in soup.find(itemprop = 'streetAddress').text.split('\n') if item) if item][0]
        except:
            print('street address field missing')
            row_results['addr'] = 'missing'
        try:
            row_results['city'] = soup.find(itemprop = 'addressLocality').text
        except:
            row_results['city'] = 'missing'
            print('city field missing')
        try:
            row_results['zip'] = soup.find(itemprop = 'postalcode').text
        except:
            print('zip field missing')
            riw_results['zip'] = 'missing'
        #row_results['header_info'] = ', '.join([item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item])
        row_results['name'] = [item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][0]
        row_results['degree'] = ', '.join([item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][1:-1])
        row_results['verified'] = [item.strip(',') for item in (item.strip() for item in soup.find(id = 'profHdr').text.split('\n') if item) if item][-1]
        row_results['phone'] = soup.find(class_ = "phone-number").text
        row_results['statement'] = ''.join([item for item in soup.find(class_ = 'section profile-personalstatement').text.split('\n') if item])
        finances = [item for item in (item.strip() for item in soup.find(id = 'tabs-finances-office').text.split('\n') if item) if item]
        finances.remove(finances[-1])
        print(finances)
        
        if any("Cost per Session" in s for s in finances):
            for i in range(len(finances)):
                if 'Cost per Session: ' in finances[i]:
                    row_results['cost_per_ses'] = finances[i].replace('Cost per Session: ', '')
        else:
            print('Cost per Session field missing')
            row_results['cost_per_ses'] = 'missing cost_per_ses'
            
        if any('Sliding Scale' in s for s in finances):
            for i in range(len(finances)):
                if 'Sliding Scale: ' in finances[i]:
                    row_results['sliding_scale'] = finances[i].replace('Sliding Scale: ', '')
        else:
            print('Sliding Scale field missing')
            row_results['sliding_scale'] = 'missing sliding_scale'
            
        for i in range(len(finances)):
            if 'Pay By' in finances[i]:
                try:
                    row_results['pay_by'] = finances[i+1]
                except: 
                    print('missing pay_by')
                    row_results['pay_by'] = 'missing pay by'
                    
        if 'Accepted Insurance Plans' not in finances:
                print('insurance field missing')
                row_results['accepted_insurance'] = 'missing insurance'
        else:
            for i in range(len(finances)):
                if 'Accepted Insurance Plans' in finances[i]:
                    row_results['accepted_insurance'] = [i for i in finances[i+1:]]
        #row_results['quals'] = soup.find(class_ = 'profile-qualifications details-section top-border').text
        check = ['Years in Practice:','School:',  'Year Graduated:', 'License and State:']
        quals = soup.find(class_ = 'profile-qualifications details-section top-border').text
        quals_list = [item for item in (item.strip() for item in quals.split('\n') if item) if item][1:]
        if len(quals_list) > 8:
            print('check this link')
        for i in check:
            if i in quals_list:
                row_results[i] = quals_list[quals_list.index(i)+1]
            else:
                row_results[i] = 'missing'
                print(i + ' field missing')
        row_results['specs'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-top').text.split('\n') if item) if item])
        row_results['issues'] = ', '.join([item for item in (item.strip() for item in soup.find(class_= 'spec-list attributes-issues').text.split('\n') if item) if item][1:])
        try:
            row_results['mental_health'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-mental-health').text.split('\n') if item) if item][1:])
            
        except:
            print('mental health field missing')
            row_results['mental_health'] = 'missing'
        try:
            row_results['sexuality'] = ', '.join([item for item in (item.strip() for item in soup.find(class_ = 'spec-list attributes-sexuality').text.split('\n') if item) if item][1:])
        except:
            print('sexuality field missing')
            row_results['sexuality'] = 'missing'
        try:  
            row_results['client_focus_age'] = soup.find(class_= 'spec-list attributes-age-focus').text
        except:
            print('client focus age field missing')
            row_results['client_focus_age'] = 'missing'
        try:
            row_results['client_focus_communities'] = soup.find(class_ = 'spec-list attributes-categories').text
        except:
            print('client focus communities field missing')
            row_results['client_focus_communities'] = 'missing'
        try:
            row_results['treatment_approach_types_of_ther'] = soup.find(class_= 'spec-list attributes-treatment-orientation').text
        except:
            print('treatment approach types of therapy field missing')
            row_results['treatment_approach_types_of_ther'] = 'missing'
        try:
            row_results['treatment_approach_modality'] = soup.find(class_ = 'spec-list attributes-modality').text
        except:
            print('treatment approach modality field missing')
            row_results['treatment_approach_modality'] = 'missing'
        try:
            row_results['additional_creds'] = soup.find(class_ = 'profile-additional-credentials details-section top-border').text
        except:
            print('additional credentials field missing')
            row_results['additional_creds'] = 'missing'
            
        driver.quit()
        final_df = final_df.append(row_results, ignore_index=True)
        print('---'*10)
        
        

In [13]:
scraper(urls, path)

['Cost per Session: $275 - $325', 'Sliding Scale: Yes', 'Pay By:', 'American Express, Cash, Check, Health Savings Account, Mastercard, Visa', 'Accepted Insurance Plans', 'Aetna', 'Blue Cross', 'Blue Shield', 'BlueCross BlueShield of Western New York', 'BlueCross and BlueShield', 'BlueShield of Northeastern New York', 'Cigna', 'Empire BlueCross BlueShield', 'Excellus BlueCross BlueShield', 'Oxford', 'UnitedHealthcare', 'Out of Network']
------------------------------
street address field missing
['Sliding Scale: Yes', 'Pay By:']
Cost per Session field missing
missing pay_by
insurance field missing
Years in Practice: field missing
Year Graduated: field missing
mental health field missing
sexuality field missing
client focus communities field missing
treatment approach modality field missing
additional credentials field missing
------------------------------
['Cost per Session: $80 - $200', 'Sliding Scale: Yes', 'Pay By:', 'Mastercard, Visa', 'Accepted Insurance Plans', 'Aetna', 'BlueCros

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
final_df

,name,verified,degree,addr,city,state,zip,phone,cost_per_ses,sliding_scale,pay_by,accepted_insurance,specs,issues,mental_health,sexuality,client_focus_age,client_focus_communities,treatment_approach_types_of_ther,treatment_approach_modality,additional_creds,License and State:,School:,Year Graduated:,Years in Practice:,statement
0,Kristie Tse,Verified by Psychology Today,"Counselor, MA, LMHC, NCC",155 E 44th St,"New York,",New York,10017,(646) 480-9561,$275 - $325,Yes,"American Express, Cash, Check, Health Savings ...","[Aetna, Blue Cross, Blue Shield, BlueCross Blu...","Mood Disorders, Transgender, Substance Use","ADHD, Addiction, Anger Management, Antisocial ...","Dissociative Disorders, Personality Disorders","Bisexual, Gay, Lesbian",\nAge\n\n\n\n ...,\nCommunities\n\n\n\n ...,\nTypes of Therapy\n\n\n\nCognitive Behavioral...,\nModality\n\n\n\n ...,\nAdditional Credentials\n\nCertificate: Natio...,009672 New York,New York University,2017,4 Years,Amidst the chaos and pressure of living in the...
1,Dayna Shapiro,Verified by Psychology Today,"Psychologist, PsyD",missing,"New York,",New York,10001,(917) 472-1234,missing cost_per_ses,Yes,missing pay by,missing insurance,"Depression, Anxiety, Trauma and PTSD","Child or Adolescent, Coping Skills, Family Con...",missing,missing,\nAge\n\n\n\n ...,missing,\nTypes of Therapy\n\n\n\nAttachment-based\n\n...,missing,missing,022319 New York,Pace University,missing,missing,Are you feeling overwhelmed because you are go...
2,Marlene M Reil,Verified by Psychology Today,"Psychologist, PhD, CASAC","303 Fifth Avenue, Suite 1407","New York,",New York,10016,(646) 553-6310,$80 - $200,Yes,"Mastercard, Visa","[Aetna, BlueCross and BlueShield]","Substance Use, Depression, Weight Loss","Addiction, Alcohol Use, Anger Management, Anxi...","Dissociative Disorders, Elderly Persons Disord...","Bisexual, Gay, Lesbian",\nAge\n\n\n\n ...,\nCommunities\n\n\n\n ...,\nTypes of Therapy\n\n\n\nCoaching\n\n\nCognit...,\nModality\n\n\n\n ...,missing,019040 New York,New School Univeristy,2006,missing,Life in the twenty-first century challenges us...
3,Kelli McGinness,Verified by Psychology Today,"Clinical Social Work/Therapist, LMSW",153 West 27th Street,"New York,",New York,10001,(646) 783-1015,missing cost_per_ses,Yes,"American Express, Cash, Check, Discover, Maste...",[Out of Network],"Addiction, Anxiety, Trauma and PTSD","Behavioral Issues, Chronic Relapse, Codependen...",Personality Disorders,"Bisexual, Gay, Lesbian",missing,missing,missing,missing,\nAdditional Credentials\n\nCertificate: New J...,108133 New York,Rutgers University,2016,missing,"A firm believer in reciprocity, I believe ther..."


### IGNORE THIS SECTION!! just experimenting and testing here

In [5]:
driver = Chrome(r'C:/Users/Tinatin/.wdm/drivers/chromedriver/win32/83.0.4103.39/chromedriver.exe')
driver.get('https://www.psychologytoday.com/us/therapists/african-american/ny/new-york/345502?sid=5eee3e019ee48&ref=9&tr=ResultsPhoto')

In [6]:
soup = BeautifulSoup(driver.page_source,'html.parser') 

In [16]:
clint_focus_age = soup.find(class_= 'spec-list attributes-age-focus').text

In [17]:
client_focus_communities = soup.find(class_ = 'spec-list attributes-categories').text

In [18]:
treatment_approach_types_of_ther = soup.find(class_= 'spec-list attributes-treatment-orientation').text

In [19]:
treatment_approach_modality = soup.find(class_ = 'spec-list attributes-modality').text

In [20]:
soup.find(class_ = 'profile-additional-credentials details-section top-border').text

'\nAdditional Credentials\n\nCertificate: National Board for Certified Counselors / 1214994\nCertificate Date: 2019\n\n'

In [27]:
check = []
for i in soup.findAll(class_= 'specialties-section top-border hidden-md-up'):
    check.append(i.findAll('a'))

In [31]:
for icheck[-1]:
    print(i.text)

New York, NY
New York
10018
10017
10016


In [32]:
len(check[-1])

5